## Regex with Python


In [44]:
import re
import spacy
from spacy.tokens import Span

In [45]:
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

In [46]:
regex_pattern = r"Paul [A-Z]\w+"

In [47]:
matches = re.finditer(regex_pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [48]:
nlp = spacy.blank("en")

In [49]:
doc = nlp(text)
original_ents = list(doc.ents)
mwt_ents = []

for match in re.finditer(regex_pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label = "PERSON")
    original_ents.append(per_ent)

doc.ents = original_ents

for ent in doc.ents:
    print(ent.text, ent.label_)
        

Paul Newman PERSON
Paul Hollywood PERSON


In [50]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [51]:
from spacy.language import Language

@Language.component("doruk_ner")
def doruk_ner(doc):
    regex_pattern = r"Paul [A-Z]\w+"
    original_ents = list(doc.ents)
    mwt_ents = []

    for match in re.finditer(regex_pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label = "PERSON")
        original_ents.append(per_ent)

    doc.ents = original_ents

    for ent in doc.ents:
        print(ent.text, ent.label_)
    
    return(doc)
    

In [52]:
nlp2 = spacy.blank("en")
nlp2.add_pipe("doruk_ner")

doc2 = nlp2(text)
print(doc2.ents)

Paul Newman PERSON
Paul Hollywood PERSON
(Paul Newman, Paul Hollywood)


In [68]:
@Language.component("holly_cinema_ner")
def cinema_ner(doc):
    regex_pattern = r"Hollywood"
    original_ents = list(doc.ents)
    mwt_ents = []

    for match in re.finditer(regex_pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label = "CINEMA")
        original_ents.append(per_ent)
        
    
    filtered = spacy.util.filter_spans(original_ents)

    doc.ents = filtered
    return(doc)

In [69]:
nlp3 = spacy.load("en_core_web_sm")
nlp3.add_pipe("holly_cinema_ner")

<function __main__.cinema_ner(doc)>

In [70]:
doc3 = nlp3(text)
for ent in doc3.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
